# Notes

## Flight paths:
- **Houston, TX** to **Los Angeles, CA** (IAH - LAX)
- **New York City, NY** to **Miami, FL** (JFK - MIA)
- **Portland, WA** to **Chicago, IL** (PDX - ORD)

## Number of Total Routes
- at least 1,000 per route for now (All times in GMC)

    -__Times:__

        - 0000 hours to 0600 hours

        - 0601 hours to 1200 hours

        - 1201 hours to 1800 hours

        - 1801 hours to 2399 hours

    - __Times of the Year:__
    
        - Try to get every month

    - __Times of the Week:__
        - Try to get every day. 
    
## Notes about Data considerations

- see what are typical flight times for your paths. you may be limited here. 

- consistently work with other api to see what you can grab

- grab future flights too!

- do some division on how many flights you can grab from how many time zonestuff


## Features to Scrape
- want aircraft type
- want airline flight info
- want airline flight 
- want aircraft type struct
- want flight struct

1 query will generate 15 results. Ex; If you request to see all flight from airport Alpha to Airport Bravo and the search results come back with 5000 flights. To find the pricing estimate you would do the following math. 5000/15 = 333 * $0.0079 = $2.63 (Class 2)

In [269]:
import sys
from suds import null, WebFault
from suds.client import Client
import logging
import json
import pandas as pd
import datetime
import numpy as np

In [2]:
with open('/Users/ChristopherKuzemka/Documents/GA/dsi_11/projects/capstone/env.json') as f:
    information = json.load(f)

In [3]:
information.keys()

dict_keys(['FA_API_KEY', 'FA_USERNAME', 'x-rapidapi-host', 'x-rapidapi-key'])

In [4]:
username = information.get('FA_USERNAME')
apiKey = information.get('FA_API_KEY')
url = 'http://flightxml.flightaware.com/soap/FlightXML2/wsdl'

In [5]:
logging.basicConfig(level=logging.INFO)
api = Client(url, username=username, password=apiKey)

## Testing playground determining usefulness of API

In [83]:
#Get Aircraft type  - semi-useful
aircraft_type = api.service.AircraftType('B744')
aircraft_type

(AircraftTypeStruct){
   manufacturer = "Boeing"
   type = "747-400"
   description = "quad-jet"
 }

# JFK - MIA

From [here](https://www.flights.com/flights/new-york-jfk-to-miami-mia/): "with 3 differnt airlines operating flights between New York and Miami, there are, on average, 2,197 flights per month.. This equates to about 523 flights per week, and 75 flights per day from JFK to MIA. The three airlines are:
- American Airlines (Flight AA 2572)

- British Airways (Flight BA 1687) 

- Malaysia Airlines (Flight MH 9446).

In [7]:
def make_unix_lists(start_date, end_date, frequency):
    created_range = pd.date_range(start = start_date, end = end_date, freq = frequency) #creates a daterange series
    list_created_range = list(created_range) #converts such range into a list
    unix_floats = [date.to_pydatetime().timestamp() for date in list_created_range] #transforms the daterange list into unix epoch tiimestamps represented as floats
    unix_ints = [int(i) for i in unix_floats] #makes the above list as a list of integers
    #return unix_ints

    #We are doing the below to accomodate a for loop format into another function
    start_ints = unix_ints[:-1] #creates a list of all the start dates without last element
    end_ints = unix_ints[1:] #creates a list of all the end dates without first element 

    return start_ints, end_ints

In [8]:
start, end = make_unix_lists('5/11/2020', '5/24/2020', '8H')

In [9]:
start

[1589169600,
 1589198400,
 1589227200,
 1589256000,
 1589284800,
 1589313600,
 1589342400,
 1589371200,
 1589400000,
 1589428800,
 1589457600,
 1589486400,
 1589515200,
 1589544000,
 1589572800,
 1589601600,
 1589630400,
 1589659200,
 1589688000,
 1589716800,
 1589745600,
 1589774400,
 1589803200,
 1589832000,
 1589860800,
 1589889600,
 1589918400,
 1589947200,
 1589976000,
 1590004800,
 1590033600,
 1590062400,
 1590091200,
 1590120000,
 1590148800,
 1590177600,
 1590206400,
 1590235200,
 1590264000]

In [10]:
jfk_mia_info = {'AA':'2572',
                'BA':'1687',
                'MA':'9446'}

In [380]:
def get_flights_df(start_input, end_input, frequency_input, origin_input, destination_input, flight_info_input, howMany_input):
    ## Empty lists for storage reasons
    schedules = [] #instantiates the schedules column
    modified_schedules = [] #instantiates the modified schedules list
    flight_info_list = [] #instantiates the flight info list


    ## Creates the sketelton dataframe for schedules_df
    schedules_cols = ['ident', 'actual_ident', 'departuretime', 'arrival_time', 'origin', 'destination', 'aircrafttype', 'meal_service', 'seats_cabin_first', 'seats_cabin_business', 'seats_cabin_coach'] #column names for the scheduling df
    schedules_df = pd.DataFrame(columns = column_names) #creates the empty schedules dataframe skeleton

    ## Creates the skeleton dataframe for the flight_info_df
    flight_info_cols = ['faFlightID', 'ident', 'aircrafttype', 'filed_ete', 'filed_time', 'filed_departuretime', 'filed_airspeed_mach', 'filed_altitude', 'actualdeparturetime', 'estimatedarrivaltime', 'actualarrivaltime', 'diverted', 'origin', 'originName', 'originCity', 'destinationName', 'destinationCity']
    flight_info_df = pd.DataFrame(columns = names) #creates the empty flight_info dataframe skeleton

    ## Unix Epoch Time Function
    start_list, end_list = make_unix_lists(start_input, end_input, frequency_input) #utilizes the unix epoch creation list function

    ## Magic happens here
    for airline_code in flight_info_input: #for the keys in the flight info input 
        for j in range(len(start_list)): #for the entirity of the unix epoch time list
            airline_flight_schedules = api.service.AirlineFlightSchedules(startDate = start_list[j], endDate = end_list[j], origin = origin_input, destination = destination_input, airline = airline_code, flightno = flight_info_input.get(airline_code), howMany = howMany_input) #gets the flight schedules information

            #Credit for the dict below
            #https://stackoverflow.com/questions/17581731/parsing-suds-soap-complex-data-type-into-python-dict
            airline_flight_dict = Client.dict(airline_flight_schedules) #converts the above variable to a dictionary
            schedules.append(airline_flight_dict) #appends results to the instantiated output list
    
    ## Cleans the dirty collection for schedules
    for k in range(len(schedules)):
        try: #try the below...we'll get some errors for empty stuff
            modified_schedules.append(schedules[k]['data']) #filters the output list to only show meaningful information where actual flights were flows in a timeframes studied
        except: #when an error occurs...
            continue #..move on and ignore any issues with with non existent flight data

    ## Creates a dataframe out of the modified_schedules list <-- FIX THIS LATER
    for l in range(len(modified_schedules)):
        for m in range(len(modified_schedules[l])):
            sched_df = pd.DataFrame(modified_schedules[l][m]).T #uses a transposed version to cleanly extract the data
            sched_df.rename(columns = {0:schedules_cols[0], #renames the columns to match the end_df so concatenation is simple
                        1: schedules_cols[1],
                        2: schedules_cols[2],
                        3: schedules_cols[3],
                        4: schedules_cols[4],
                        5: schedules_cols[5],
                        6: schedules_cols[6],
                        7: schedules_cols[7],
                        8: schedules_cols[8],
                        9: schedules_cols[9],
                        10: schedules_cols[10]}, inplace = True) #very inefficient way to rename my columns
            schedules_df = pd.concat([schedules_df, sched_df]) #overwrites the end_df to create a larger dataframe
    
    ## Cleans the schedules_df
    schedules_df.drop(index = 0, inplace = True) #drops the 0 index rows in the repetative 0,1 pattern of the dataframe
    schedules_df.reset_index(inplace = True) #resets the index by creating a new index row
    schedules_df.drop(columns = 'index', inplace = True) #drops the extra 'index row
    schedules_df.drop_duplicates(subset = ['arrival_time'], inplace = True) #drops the duplicate time epochs so that we don't have impossible situations of the same flight coming into the same place at the same exact time


    ## Gets the flight information list using the schedules_df idents
    for identifier in schedules_df['ident']: #calls to each identifier collected
        for offset_number in range(15): #set in place to not miss any flights in 2 week period 
            try: #we're gonna get some errors here for empty flights I think
                flight_info_ex = api.service.FlightInfoEx(ident = identifier, howMany = 15, offset = offset_number) #retrieves the information pertaining to the identifier
                flight_info = Client.dict(flight_info_ex) #converts the suds object into a workable dictionary
                flight_info_list.append(flight_info_ex) #appends whatever the above is into a list
            except: #when an error occurs...
                continue #...just move on

    ## Creates a dataframe out of the flight_info_df
    ## Credit to Alex Speicher from Hofstra University for showing an efficient way to do this
    for query in flight_info_list: #calls to every query in the info list made
        for flight in query['flights']: #for every flight in the dictionary
            flight_dict = {} #empty dicitonary pertaining to the columns in each flight 
            for column in flight_info_cols: #for every column name in the flight_info_cols
                flight_dict[column] = flight[column] #appends the column names to the dictionary
            info_df = pd.DataFrame([flight_dict], columns=flight_dict.keys()) #makes a dataframe using the dictionary columns 
            flight_info_df = pd.concat([flight_info_df, info_df], axis =0).reset_index(drop = True) #resets the index and fixes all those issues

    #Cleans the flight_info_df
    flight_info_df.drop_duplicates(subset = ['faFlightID'], inplace = True) #eliminates the overlapping most unique identifiers in our
    flight_info_df = flight_info_df[flight_info_df['actualarrivaltime'] != 0] #removes the currently active flights from the list as there is no data on them and overwrites the dataframe variable
    flight_info_df.reset_index(inplace = True) #resets the index 

    return schedules_df, flight_info_df #returns two dataframes

In [381]:
jfk_mia_flight_scheds, jfk_mia_flight_info = get_flights_df('5/15/2020', '5/25/2020', '8H', 'JFK', 'MIA', jfk_mia_info, 15)

ERROR:suds.client:<suds.sax.document.Document object at 0x12099a110>
ERROR:suds.client:<suds.sax.document.Document object at 0x117c7c710>
ERROR:suds.client:<suds.sax.document.Document object at 0x1202f7510>
ERROR:suds.client:<suds.sax.document.Document object at 0x120b27150>
ERROR:suds.client:<suds.sax.document.Document object at 0x122c703d0>
ERROR:suds.client:<suds.sax.document.Document object at 0x1070bdd90>
ERROR:suds.client:<suds.sax.document.Document object at 0x1221c2750>
ERROR:suds.client:<suds.sax.document.Document object at 0x122291710>
ERROR:suds.client:<suds.sax.document.Document object at 0x12212f810>
ERROR:suds.client:<suds.sax.document.Document object at 0x124108650>
ERROR:suds.client:<suds.sax.document.Document object at 0x125947610>
ERROR:suds.client:<suds.sax.document.Document object at 0x12626c190>
ERROR:suds.client:<suds.sax.document.Document object at 0x125267650>
ERROR:suds.client:<suds.sax.document.Document object at 0x12533cad0>
ERROR:suds.client:<suds.sax.docume

In [514]:
jfk_mia_flight_scheds

,ident,actual_ident,departuretime,arrival_time,origin,destination,aircrafttype,meal_service,seats_cabin_first,seats_cabin_business,seats_cabin_coach
0,MAS9446,AAL2572,1589567400,1589579280,KJFK,KMIA,B738,Business: Meal / Economy: Meal,0,16,144
1,GLO6391,AAL2572,1589569200,1589581200,KJFK,KMIA,B738,Business: No meal / Economy: No meal,0,16,144
3,MAS9446,AAL2572,1589653800,1589665680,KJFK,KMIA,B738,Business: Meal / Economy: Meal,0,16,144
4,GLO6391,AAL2572,1589655600,1589667600,KJFK,KMIA,B738,Business: No meal / Economy: No meal,0,16,144
6,MAS9446,AAL2572,1589740200,1589752080,KJFK,KMIA,B738,Business: Meal / Economy: Meal,0,16,144
7,GLO6391,AAL2572,1589742000,1589754000,KJFK,KMIA,B738,Business: No meal / Economy: No meal,0,16,144
9,MAS9446,AAL2572,1589826600,1589838480,KJFK,KMIA,B738,Business: Meal / Economy: Meal,0,16,144
10,GLO6391,AAL2572,1589828400,1589840400,KJFK,KMIA,B738,Business: No meal / Economy: No meal,0,16,144
12,MAS9446,AAL2572,1589913000,1589924880,KJFK,KMIA,B738,Business: Meal / Economy: Meal,0,16,144
13,AAL2572,None,1589914800,1589926800,KJFK,KMIA,B738,Business: No meal / Economy: No meal,0,16,144


In [366]:
jfk_mia_flight_info.drop_duplicates(subset = ['faFlightID'], inplace = True)

In [369]:
mask = jfk_mia_flight_info['actualarrivaltime'] != 0
jfk_mia_flight_info[mask]

,faFlightID,ident,aircrafttype,filed_ete,filed_time,filed_departuretime,filed_airspeed_mach,filed_altitude,actualdeparturetime,estimatedarrivaltime,actualarrivaltime,diverted,origin,originName,originCity,destinationName,destinationCity
3,AAL2572-1590209159-airline-0537,AAL2572,B738,03:06:00,1590431725,1590432000,None,360,1590431773,1590440887,1590440887,None,KJFK,John F Kennedy Intl,"New York, NY",Miami Intl,"Miami, FL"
4,AAL2572-1590122721-airline-0420,AAL2572,B738,02:40:00,1590345342,1590345600,None,380,1590345390,1590354360,1590354360,None,KJFK,John F Kennedy Intl,"New York, NY",Miami Intl,"Miami, FL"
5,AAL2572-1590036344-airline-0175,AAL2572,B738,02:55:00,1590259235,1590259200,None,380,1590259283,1590267480,1590267480,None,KJFK,John F Kennedy Intl,"New York, NY",Miami Intl,"Miami, FL"
6,AAL2572-1589949964-airline-0120,AAL2572,B738,02:40:00,1590172660,1590172800,None,380,1590172708,1590181620,1590181620,None,KJFK,John F Kennedy Intl,"New York, NY",Miami Intl,"Miami, FL"
7,AAL2572-1589863519-airline-0459,AAL2572,B738,02:40:00,1590086190,1590086400,None,380,1590086238,1590095340,1590095340,None,KJFK,John F Kennedy Intl,"New York, NY",Miami Intl,"Miami, FL"
8,AAL2572-1589777123-airline-0485,AAL2572,B738,02:38:00,1590000375,1590000000,None,380,1590000407,1590009900,1590009900,None,KJFK,John F Kennedy Intl,"New York, NY",Miami Intl,"Miami, FL"
9,AAL2572-1589690730-airline-0195,AAL2572,B738,02:46:00,1589914315,1589913600,None,380,1589914363,1589924160,1589924160,None,KJFK,John F Kennedy Intl,"New York, NY",Miami Intl,"Miami, FL"
10,AAL2572-1589604351-airline-0000,AAL2572,B738,02:41:00,1589827142,1589827200,None,320,1589827191,1589837040,1589837040,None,KJFK,John F Kennedy Intl,"New York, NY",Miami Intl,"Miami, FL"
11,AAL2572-1589517913-airline-0098,AAL2572,B738,02:40:00,1589741446,1589740800,None,380,1589741494,1589750762,1589750762,None,KJFK,John F Kennedy Intl,"New York, NY",Miami Intl,"Miami, FL"
12,AAL2572-1589431536-airline-0436,AAL2572,B738,02:40:00,1589654365,1589654400,None,300,1589654430,1589663460,1589663460,None,KJFK,John F Kennedy Intl,"New York, NY",Miami Intl,"Miami, FL"


In [378]:
mask = jfk_mia_flight_scheds.duplicated(subset = ['arrival_time']) == True
jfk_mia_flight_scheds[mask]

,ident,actual_ident,departuretime,arrival_time,origin,destination,aircrafttype,meal_service,seats_cabin_first,seats_cabin_business,seats_cabin_coach
2,AAL2572,None,1589569200,1589581200,KJFK,KMIA,B738,Business: No meal / Economy: No meal,0,16,144
5,AAL2572,None,1589655600,1589667600,KJFK,KMIA,B738,Business: No meal / Economy: No meal,0,16,144
8,AAL2572,None,1589742000,1589754000,KJFK,KMIA,B738,Business: No meal / Economy: No meal,0,16,144
11,AAL2572,None,1589828400,1589840400,KJFK,KMIA,B738,Business: No meal / Economy: No meal,0,16,144
16,AAL2572,None,1590001200,1590013200,KJFK,KMIA,B738,Business: No meal / Economy: No meal,0,16,144
19,AAL2572,None,1590087600,1590099600,KJFK,KMIA,B738,Business: No meal / Economy: No meal,0,16,144
22,AAL2572,None,1590174000,1590186000,KJFK,KMIA,B738,Business: No meal / Economy: No meal,0,16,144
25,AAL2572,None,1590260400,1590272400,KJFK,KMIA,B738,Business: No meal / Economy: No meal,0,16,144
28,AAL2572,None,1590346800,1590358800,KJFK,KMIA,B738,Business: No meal / Economy: No meal,0,16,144


In [611]:
def get_flights(start_input, end_input, frequency_input, destination_input):
    ## Unix Epoch Time Function
    start_list, end_list = make_unix_lists(start_input, end_input, frequency_input) #utilizes the unix epoch creation list function
    
    ## Create an empty list to add searches to
    get_flights_list = []
    schedules = []
    modified_schedules = []    


    ## Skeleton df for get_flights_df
    get_flights_cols = ['faFlightID', 'ident', 'prefix', 'type', 'suffix', 'origin', 'destination', 'timeout', 'timestamp', 'departureTime', 'firstPositionTime', 'arrivalTime', 'longitude', 'latitude', 'lowLongitude', 'lowLatitude', 'highLongitude', 'highLatitude', 'groundspeed', 'altitude', 'heading', 'altitudeStatus', 'updateType', 'altitudeChange', 'waypoints']
    get_flights_df = pd.DataFrame(columns = get_flights_cols)

    ## Creates the sketelton dataframe for schedules_df
    schedules_cols = ['ident', 'actual_ident', 'departuretime', 'arrival_time', 'origin', 'destination', 'aircrafttype', 'meal_service', 'seats_cabin_first', 'seats_cabin_business', 'seats_cabin_coach'] #column names for the scheduling df
    schedules_df = pd.DataFrame(columns = column_names) #creates the empty schedules dataframe skeleton


    print("Grabbing In-Flight Information...")
    #Grabbing flights from the list of destinations
    for destination in destination_input:
        add_dest = '{= dest ' + destination + '}'
        search_flight = api.service.SearchBirdseyeInFlight(query = add_dest, howMany = 15)
        search_dict = Client.dict(search_flight)
        print("Destination Query Searched!")
        for retrieved_query in range(len(search_dict['aircraft'])):
            get_flights_list.append(search_dict['aircraft'][retrieved_query])
            


    print("Creating In-Flight Dataframe")
    #Creates the dataframe and concats to the get_flights_df
    for list_item in range(len(get_flights_list)):
        preliminary_flight_df = pd.DataFrame(get_flights_list[list_item]).T #uses a transposed version to cleanly extract the data
        preliminary_flight_df.rename(columns = {0:get_flights_cols[0], #renames the columns to match the get_flights_df so concatenation is simple
                            1: get_flights_cols[1],
                            2: get_flights_cols[2],
                            3: get_flights_cols[3],
                            4: get_flights_cols[4],
                            5: get_flights_cols[5],
                            6: get_flights_cols[6],
                            7: get_flights_cols[7],
                            8: get_flights_cols[8],
                            9: get_flights_cols[9],
                            10: get_flights_cols[10],
                            11: get_flights_cols[11],
                            12: get_flights_cols[12],
                            13: get_flights_cols[13],
                            14: get_flights_cols[14],
                            15: get_flights_cols[15],
                            16: get_flights_cols[16],
                            17: get_flights_cols[17],
                            18: get_flights_cols[18],
                            19: get_flights_cols[19],
                            20: get_flights_cols[20],
                            21: get_flights_cols[21],
                            22: get_flights_cols[22],
                            23: get_flights_cols[23],
                            24: get_flights_cols[24]}, inplace = True) #very inefficient way to rename my columns
        get_flights_df = pd.concat([get_flights_df, preliminary_flight_df]) #overwrites the get_flights_df to create a larger dataframe
        print("Conactenated In-Flight Dataframe!")

    print('Cleaning The In-Flight Dataframe...')
    ## Cleans the get_flights_df        
    get_flights_df.drop(index = 0, inplace = True) #drops the 0 index rows in the repetative 0,1 pattern of the dataframe
    get_flights_df.reset_index(inplace = True) #resets the index by creating a new index row
    get_flights_df.drop(columns = 'index', inplace = True) #drops the extra index row


    print("Grabbing Airline Schedule Information...")
    ## Gets airline schedule and flight data
    for origination in range(len(get_flights_df['origin'])): #for the keys in the flight info input
        print(f'Searching Airline Schedules For Origin: {get_flights_df.loc[origination, "origin"]}') 
        for time_epoch in range(len(start_list)): #for the entirity of the unix epoch time list
            airline_flight_schedules = api.service.AirlineFlightSchedules(startDate = start_list[time_epoch], endDate = end_list[time_epoch], origin = get_flights_df.loc[origination, 'origin'], destination = get_flights_df.loc[origination, 'destination'], howMany = 15) #gets the flight schedules information
            airline_flight_dict = Client.dict(airline_flight_schedules) #converts the above variable to a dictionary
            schedules.append(airline_flight_dict) #appends results to the instantiated output list
            print("Airline Shcedule Searched!")

    print("Cleaning Dirty Schedules Scrape...")
    ## Cleans the dirty collection for schedules
    for item in range(len(schedules)):
        try: #try the below...we'll get some errors for empty stuff
            modified_schedules.append(schedules[k]['data']) #filters the output list to only show meaningful information where actual flights were flows in a timeframes studied
        except: #when an error occurs...
            continue #..move on and ignore any issues with with non existent flight data

    
    print('Making The Schedules Dataframe...')
    ## Creates a dataframe out of the modified_schedules list <-- FIX THIS LATER
    for l in range(len(modified_schedules)):
        for m in range(len(modified_schedules[l])):
            sched_df = pd.DataFrame(modified_schedules[l][m]).T #uses a transposed version to cleanly extract the data
            sched_df.rename(columns = {0:schedules_cols[0], #renames the columns to match the schedules_df so concatenation is simple
                        1: schedules_cols[1],
                        2: schedules_cols[2],
                        3: schedules_cols[3],
                        4: schedules_cols[4],
                        5: schedules_cols[5],
                        6: schedules_cols[6],
                        7: schedules_cols[7],
                        8: schedules_cols[8],
                        9: schedules_cols[9],
                        10: schedules_cols[10]}, inplace = True) #very inefficient way to rename my columns
            schedules_df = pd.concat([schedules_df, sched_df]) #overwrites the schedules_df to create a larger dataframe
            print("Schedules Dataframe Concatenated!")

    print("Cleaning Schedules Dataframe...")
    ## Cleans the schedules_df
    #schedules_df.drop(index = 0, inplace = True) #drops the 0 index rows in the repetative 0,1 pattern of the dataframe
    schedules_df.reset_index(inplace = True) #resets the index by creating a new index row
    schedules_df.drop(columns = 'index', inplace = True) #drops the extra 'index row
    #schedules_df.drop_duplicates(subset = ['arrival_time'], inplace = True) #drops the duplicate time epochs so that we don't have impossible situations of the same flight coming into the same place at the same exact time

    return get_flights_df, schedules_df

In [603]:
destination_list = ['KLAX', 'KJFK', 'KMIA', 'KORD', 'KPDX', 'KIAH', 'KATL']
flights_df, flights_scheds = get_flights('5/1/2020','5/27/2020','8H', destination_list)

In [609]:
flights_df.loc[0, 'origin']

CYYZ

In [605]:
flights_scheds

,ident,actual_ident,departuretime,arrival_time,origin,destination,aircrafttype,meal_service,seats_cabin_first,seats_cabin_business,seats_cabin_coach


In [ ]:
for origination in test_list['origin']: #for the keys in the flight info input 
    for j in range(len(start_list)): #for the entirity of the unix epoch time list
        airline_flight_schedules = api.service.AirlineFlightSchedules(startDate = start_list[j], endDate = end_list[j], origin = origin_input, destination = destination_input, airline = airline_code, flightno = flight_info_input.get(airline_code), howMany = howMany_input) #gets the flight schedules information

        #Credit for the dict below
        #https://stackoverflow.com/questions/17581731/parsing-suds-soap-complex-data-type-into-python-dict
        airline_flight_dict = Client.dict(airline_flight_schedules) #converts the above variable to a dictionary
        schedules.append(airline_flight_dict) #appends results to the instantiated output list

In [578]:
test_list.loc[0, 'origin']

CYYZ

In [607]:
airline_flight_schedule = api.service.AirlineFlightSchedules(startDate = start[6], endDate = end[-1], origin = 'KJFK', destination = 'KMIA', howMany = 15)

In [608]:
airline_flight_schedule

(ArrayOfAirlineFlightScheduleStruct){
   next_offset = 15
   data[] = 
      (AirlineFlightScheduleStruct){
         ident = "AAL2268"
         actual_ident = None
         departuretime = 1589365800
         arrivaltime = 1589376600
         origin = "KJFK"
         destination = "KMIA"
         aircrafttype = "B738"
         meal_service = "Business: No meal / Economy: No meal"
         seats_cabin_first = 0
         seats_cabin_business = 16
         seats_cabin_coach = 144
      },
      (AirlineFlightScheduleStruct){
         ident = "BAW4362"
         actual_ident = "AAL2268"
         departuretime = 1589365800
         arrivaltime = 1589376600
         origin = "KJFK"
         destination = "KMIA"
         aircrafttype = "B738"
         meal_service = "Business: No meal / Economy: No meal"
         seats_cabin_first = 0
         seats_cabin_business = 16
         seats_cabin_coach = 144
      },
      (AirlineFlightScheduleStruct){
         ident = "KQA7014"
         actual_iden

# IAH - LAX


From [here](https://www.flights.com/flights/houston-iah-to-los-angeles-lax/): "With 6 different airlines operating between houstan and Los Angeles, there are, on average, 2,642 flights per month. This equates to about 629 flights per week, and 90 flights per day from IAH to LAX." 5 out of 6 of these airliners are: 
- United Airlines (Flight UA 302) 
- Air China (CA 7398), Azul (AD 7026)
- Asiana Airlines (OZ 9512)
- Air New Zealand (NZ 9653)